In [668]:
import numpy as np
import random
from pprint import pprint

In [669]:
class Cardan:
    def __init__(self, n: int = 4):
        self.n = n
        self.grid = None
        self.mask = None
        self.cipher = None

    def create_grid(self, shuffle: bool = False):
        quarter_size = self.n // 2
        quarter = np.arange(1, quarter_size**2+1).reshape((quarter_size, quarter_size))
        if shuffle:
            np.random.shuffle(quarter)
        up_left_quarter = quarter
        up_right_quarter = np.rot90(up_left_quarter, k=-1)
        down_right_quarter = np.rot90(up_right_quarter, k=-1)
        down_left_quarter = np.rot90(down_right_quarter, k=-1)

        up_half = np.concatenate((up_left_quarter, up_right_quarter), axis=1)
        down_half = np.concatenate((down_left_quarter, down_right_quarter), axis=1)

        self.grid = np.concatenate((up_half, down_half), axis=0)
        return self.grid

    def create_mask(self):
        stat = {}
        # for i in range(self.n):
        #     for j in range(self.n):
        #         stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]
        for i in range(self.n//2):
            for j in range(self.n//2):
                stat[self.grid[i,j]] = [(i,j)]
        for i in range(self.n//2):
            for j in range(self.n//2, self.n):
                stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]
        for i in range(self.n//2, self.n):
            for j in range(self.n//2):
                stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]
        for i in range(self.n//2, self.n):
            for j in range(self.n//2, self.n):
                stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]

        self.mask = np.zeros((self.n, self.n))
        # for i in range(self.n // 2):
        #     for j in range(self.n // 2):
        #         self.mask[i,j] = 1
        used = random.randint(0,100) % 4
        for fields in stat.values():
            i, j = fields[used%4]
            self.mask[i,j] = 1
            used += 1

        return self.mask
    
    def create_cipher(self, text: str, verbose: bool = False):
        text_i = 0
        mask = self.mask.copy()
        self.cipher = [["_" for _ in range(self.n)] for _ in range(self.n)]
        for _ in range(4):
            if verbose:
                print(*self.cipher, sep="\n")
                print("="*100)
                print(*[[int(j) for j in i] for i in mask.tolist()], sep="\n")
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        if text_i >= len(text):
                            self.cipher[i][j] = "_"
                        else:
                            self.cipher[i][j] = text[text_i]
                            text_i += 1
            mask = np.rot90(mask, k=-1)
        return self.cipher
    
    def get_cryptogram(self):
        return "".join(["".join(row) for row in self.cipher])
    
    def get_cryptogram_key(self):
        return "".join(["".join([str(int(i)) for i in row]) for row in self.mask])
    
    def set_cryptogram(self, cryptogram: str) -> list[list[str]]:
        self.n = int(len(cryptogram)**0.5)
        self.cipher = [
            list(cryptogram[self.n*i:self.n*i+self.n]) for i in range(self.n)
        ]
        return self.cipher
    
    def set_cryptogram_key(self, cryptogram_key: str) -> np.ndarray:
        self.n = int(len(cryptogram_key)**0.5)
        self.mask = np.array([
            [int(sym) for sym in cryptogram_key[self.n*i:self.n*i+self.n]] for i in range(self.n)
        ])

        return self.mask
    
    def create_mask_from_variant(self, variant: dict[int, tuple[int,int]]) -> np.ndarray:
        if len(variant) != (self.n//2)**2:
            raise ArithmeticError(f"bad digits for creating mask {len(variant)=}, {((self.n//2)**2)=}, {self.n}")
        self.mask = np.zeros((self.n, self.n))
        for position in variant.values():
            self.mask[position[0], position[1]] = 1
        return self.mask
                
    def decode_cipher(self, verbose: bool = False) -> str:
        result = ""
        mask = self.mask.copy()
        for _ in range(4):
            if verbose:
                print(*self.cipher, sep="\n")
                print(result[len(result)-self.n:len(result)])
                print(result)
                print("="*100)
                print(*[[int(j) for j in i] for i in mask.tolist()], sep="\n")
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        result += self.cipher[i][j]
            mask = np.rot90(mask, k=-1)
        return result

        

In [670]:
def rec_cryptogram(cardan: Cardan, first_substring: str, variant: dict[int, tuple[int,int]], last_find_position: tuple[int,int] = (0,-1)):
    if len(variant)*4 == cardan.n**2 or len(first_substring) == len(variant):
        return [variant]
    variants = []
    letter = first_substring[len(variant)]
    for i in range(last_find_position[0], cardan.n):
        for j in range((i==last_find_position[0])*(last_find_position[1]+1), cardan.n):
            if cardan.cipher[i][j] == letter:
                digit = cardan.grid[i,j]
                new_variant = variant.copy()
                new_variant[int(digit)] = (i,j)
                res_rec = rec_cryptogram(cardan, first_substring, variant=new_variant, last_find_position=(i,j))
                if res_rec:
                    variants += res_rec
    
    return variants

def compute_all_variants(cardan: Cardan, bad_variant: dict[int, tuple[int,int]]):
    block = (cardan.n//2)**2
    variants = [bad_variant]
    for digit in range(1, block+1):
        if bad_variant.get(digit):
            continue
        new_coords = []
        for i in range(cardan.n):
            for j in range(cardan.n):
                if cardan.grid[i,j] == digit:
                    new_coords.append((i,j))
        new_variants = []
        for variant in variants:
            for new_coord in new_coords:
                new_variant = variant.copy()
                new_variant[digit] = new_coord
                new_variants.append(new_variant)
        variants = new_variants
    return variants


def brutforce(cryptogram: str, first_substring: str, verbose: bool = False) -> list[str]:
    results = []
    cardan = Cardan()
    cardan.set_cryptogram(cryptogram)
    cardan.create_grid()

    if (cardan.n//2)**2 > len(first_substring):
        print(f"{len(first_substring)=} будет многовато вариантов, желательно длину хотя бы {((cardan.n//2)**2)=}")
    
    if (cardan.n//2)**2 < len(first_substring):
        first_substring = first_substring[:((cardan.n//2)**2)]

    variants = rec_cryptogram(cardan, first_substring, variant={})
    new_variants = []
    for variant in variants:
        if len(variant) < (cardan.n//2)**2:
            new_variants += compute_all_variants(cardan, variant)
        elif len(variant) == (cardan.n//2)**2:
            new_variants.append(variant)
    variants = new_variants
    for variant in variants:
        cardan.create_mask_from_variant(variant)
        result = cardan.decode_cipher()
        if verbose:
            print(cardan.mask)
            print(variant)
        results.append(result)
    return results
    

In [671]:
cardan = Cardan(n=4)

In [672]:
cardan.create_grid()

array([[1, 2, 3, 1],
       [3, 4, 4, 2],
       [2, 4, 4, 3],
       [1, 3, 2, 1]])

In [673]:
cardan.create_mask()

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [674]:
cardan.create_cipher("дела-пока", verbose=True)

['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
[0, 0, 0, 1]
[0, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
['_', '_', '_', 'д']
['_', 'е', '_', '_']
['л', '_', '_', 'а']
['_', '_', '_', '_']
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 1]
['_', '-', '_', 'д']
['_', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
['_', '-', '_', 'д']
['а', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']
[1, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]


[['_', '-', '_', 'д'],
 ['а', 'е', 'п', '_'],
 ['л', '_', '_', 'а'],
 ['_', 'о', '_', 'к']]

In [675]:
cardan.get_cryptogram()

'_-_даеп_л__а_о_к'

In [676]:
cardan.get_cryptogram_key()

'0001010010010000'

In [677]:
cardan.set_cryptogram("_-_даеп_л__а_о_к")

[['_', '-', '_', 'д'],
 ['а', 'е', 'п', '_'],
 ['л', '_', '_', 'а'],
 ['_', 'о', '_', 'к']]

In [678]:
cardan.set_cryptogram_key("0001010010010000")

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 1],
       [0, 0, 0, 0]])

In [679]:
cardan.decode_cipher(verbose=True).replace("_", "")

['_', '-', '_', 'д']
['а', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']


[0, 0, 0, 1]
[0, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
['_', '-', '_', 'д']
['а', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']
дела
дела
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 1]
['_', '-', '_', 'д']
['а', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']
-пок
дела-пок
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 1, 0]
[1, 0, 0, 0]
['_', '-', '_', 'д']
['а', 'е', 'п', '_']
['л', '_', '_', 'а']
['_', 'о', '_', 'к']
а___
дела-пока___
[1, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]


'дела-пока'

In [680]:
results = brutforce(
    cryptogram="_-_даеп_л__а_о_к",
    first_substring="дел",
    verbose=True
)

len(first_substring)=3 будет многовато вариантов, желательно длину хотя бы ((cardan.n//2)**2)=4
[[0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]]
{1: (0, 3), 4: (1, 1), 2: (2, 0), 3: (0, 2)}
[[0. 0. 0. 1.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]]
{1: (0, 3), 4: (1, 1), 2: (2, 0), 3: (1, 0)}
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]]
{1: (0, 3), 4: (1, 1), 2: (2, 0), 3: (2, 3)}
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
{1: (0, 3), 4: (1, 1), 2: (2, 0), 3: (3, 1)}


In [681]:
results

['_дел-пак___о_а__',
 'даел-_пк__а___о_',
 'дела-пока_______',
 'дело-апк______а_']